In [1]:
from ipynb.fs.full.metrics import exos_running_time_df
from ipynb.fs.full.metrics import compute_performance
from ipynb.fs.full.metrics import unpickled_results
import pandas as pd
pd.set_option('display.precision', 2)

import pickle
import os

# Accuracy and Running Time

In [2]:
def get_performance_df(n_streams=2, 
                       bfname = '100K_Case1', 
                       gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/two',
                       rel_path =  'pickles/nstreams/2',
                       n_experiments=10):
    cwd = os.getcwd()
    gt_filename = f'{n_streams}_{bfname}.pkl'
    result_folder = os.path.join(cwd, rel_path)
    n_experiments = 10
    results = list()
    experiments = list()
    simulation_times = list()
    for i in range(1,n_experiments+1):
        matched, n_outliers, s_time = compute_performance(gt_folder= gt_folder, 
                                                         gt_filename= gt_filename, 
                                                         result_folder=result_folder, 
                                                         result_filename=f'{i}_{gt_filename}',
                                                         n_streams=n_streams, 
                                                         window_size=1000,
                                                         score_precision = 0.1)
        results.append((len(matched)/n_outliers) * 100)
        simulation_times.append(s_time)
        experiments.append(i)
    accuracy = {'experiment' : experiments, 'accuracy' : results, 'running_time' : simulation_times}
    df = pd.DataFrame(accuracy)
    df.to_pickle(f'{result_folder}/performance_{n_streams}_k1_{bfname}.pkl')
    return df

In [3]:
# df = get_performance_df(n_streams=30, 
#                        bfname = '100K_Case1', 
#                        gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/30',
#                        rel_path =  'pickles/nstreams/30',
#                        n_experiments=10)
# df

In [4]:
bname = 'k1_100K_Case1'
rel_path = 'pickles/nstreams'
df5 = pd.read_pickle(f'{rel_path}/5/performance_5_{bname}.pkl')
df5

,experiment,accuracy,running_time
0,1,70.68,35.93
1,2,72.48,36.87
2,3,75.22,36.06
3,4,72.68,36.25
4,5,72.96,35.52
5,6,70.40,35.79
6,7,75.68,35.42
7,8,72.66,35.90
8,9,77.08,35.98
9,10,74.58,36.51


In [5]:
df10 = pd.read_pickle(f'{rel_path}/10/performance_10_{bname}.pkl')
df10

,experiment,accuracy,running_time
0,1,75.02,57.18
1,2,77.67,57.40
2,3,75.17,57.31
3,4,76.90,56.80
4,5,76.36,57.12
5,6,75.81,56.13
6,7,76.86,55.97
7,8,76.51,55.67
8,9,76.77,58.30
9,10,75.04,58.49


In [6]:
df15 = pd.read_pickle(f'{rel_path}/15/performance_15_{bname}.pkl')
df15

,experiment,accuracy,running_time
0,1,80.97,76.30
1,2,79.87,77.82
2,3,80.34,77.19
3,4,79.64,76.95
4,5,80.67,77.99
5,6,79.37,77.04
6,7,79.38,77.27
7,8,79.35,77.34
8,9,79.21,76.86
9,10,81.03,76.75


In [7]:
df20 = pd.read_pickle(f'{rel_path}/20/performance_20_{bname}.pkl')
df20

,experiment,accuracy,running_time
0,1,82.99,108.09
1,2,82.30,108.24
2,3,82.46,107.73
3,4,80.77,108.36
4,5,81.56,108.59
5,6,80.25,108.65
6,7,81.34,108.53
7,8,82.05,109.12
8,9,82.79,108.15
9,10,81.94,108.59


In [8]:
df25 = pd.read_pickle(f'{rel_path}/25/performance_25_{bname}.pkl')
df25

,experiment,accuracy,running_time
0,1,81.84,132.41
1,2,81.81,131.67
2,3,82.46,131.80
3,4,81.62,131.80
4,5,81.65,131.87
5,6,82.27,133.24
6,7,82.58,132.97
7,8,82.41,134.40
8,9,81.10,133.12
9,10,81.74,131.74


In [9]:
df30 = pd.read_pickle(f'{rel_path}/30/performance_30_{bname}.pkl')
df30

,experiment,accuracy,running_time
0,1,83.47,158.95
1,2,83.71,160.24
2,3,83.28,160.02
3,4,83.40,159.04
4,5,82.65,161.66
5,6,82.61,161.57
6,7,83.23,161.97
7,8,82.38,163.22
8,9,81.86,170.57
9,10,83.38,162.02
